## This Notebook will serve as Data Cleaning and EDA for the weather dataset.

D02. Explanation of abbreviations & units of measurements
Metadata.csv: 
	ID = Station's ID (ex. LC-XXX) [-]
	LAT = latitute (rounded to 3 decimals for privacy reasons) [-]
	LON = longitude (rounded to 3 decimals for privacy reasons) [-]
	ALT = altitude above sea level [m]
	LCZ = local climate zone [-]. For details: see accompanying publictation.
	LC = main landcover directly around weather station (0 = impervious/1= green) [-]
	building_height = mean building height in circular buffer 10m around weather station [m]
	SVF = mean sky view factor in circular buffer 10m around weather station [-]
	installation_height = installation height of weather station [m]

LC_YYYYQX.csv: 
	DATEUTC = Date and time (UTC) in POSIXct format: YYYY-MM-DD HH:MM:SS
	ID = Station's ID (ex. LC-XXX) [-]
 	LC_HUMIDITY = Relative humidity [%]
	LC_DWPTEMP = Dew point temperature [∞C]
	LC_n = Number of 16 second observations in 10 minutes aggregate
	LC_RAD = Solar radiation [W/m2]
	LC_RAININ = Rain intensity [mm/h]
	LC_DAILYRAIN = Daily rain sum [mm]
	LC_WINDDIR = Wind direction [∞]
	LC_WINDSPEED = Wind speed [m/s]
	Date = Date in YYYY-MM-DD
	Year = Year in YYYY
	Month = Month in MM
	Day = Day in DD
	Hour = Hour in HH
	Minute = Minute in MM
	LC_RAD60 = Weighted radiation during last 60 minutes [W/m2]
	LC_TEMP_QCL0 = Temperature at QCL0 [∞C]
	LC_TEMP_QCL1 = Temperature at QCL1 [∞C]
	LC_TEMP_QCL2 = Temperature at QCL2 [∞C]
	LC_TEMP_QCL3 = Temperature at QCL3 [∞C]

In [52]:
import pandas as pd
import ydata_profiling
import sweetviz as sv
from autoviz import data_cleaning_suggestions

In [53]:
# load csv and save as dataframe
df1 = pd.read_csv('data/dataverse_files/LC_2022Q1.csv')
df2 = pd.read_csv('data/dataverse_files/LC_2022Q2.csv')
df3 = pd.read_csv('data/dataverse_files/LC_2022Q3.csv')
df4 = pd.read_csv('data/dataverse_files/LC_2022Q4.csv')
# merge all dataframes
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# load metadata and save as dataframe
df_meta = pd.read_csv('data/dataverse_files/01_Metadata_v2.csv')

In [39]:
df_meta

,ID,LAT,LON,ALT,Local_climate_zone,Landcover,building_height,Sky_view_factor,installation_height
0,LC-002,50.847,4.756,47,9,1,4.280000,0.610367,2
1,LC-003,50.870,4.728,44,5,1,5.038286,0.691315,2
2,LC-004,50.871,4.685,31,9,1,9.050964,0.798586,2
3,LC-005,50.881,4.713,26,2,1,5.050000,0.685410,2
4,LC-006,50.912,4.716,13,6,1,0.000000,0.954032,2
...,...,...,...,...,...,...,...,...,...
112,LC-134,50.926,4.725,12,9,0,14.250000,0.692417,3
113,LC-135,50.874,4.760,92,14,1,0.000000,0.716678,2
114,LC-136,50.881,4.753,44,12,1,0.000000,0.825019,2
115,LC-137,50.854,4.694,42,6,1,2.757826,0.722500,2


In [40]:
df.shape

(5546880, 21)

In [50]:
df.head()

,DATEUTC,ID,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_RAD60,LC_TEMP_QCL3
0,2022-01-01 00:10:00,2,92,11.78,38,0,0.0,0.0,-169.0,0.43,0.0,13.048027
1,2022-01-01 00:20:00,2,92,11.73,37,0,0.0,0.0,-170.0,0.33,0.0,12.985849
2,2022-01-01 00:30:00,2,92,11.73,38,0,0.0,0.0,-167.0,0.46,0.0,12.950322
3,2022-01-01 00:40:00,2,92,11.72,37,0,0.0,0.0,-160.0,0.52,0.0,12.949550
4,2022-01-01 00:50:00,2,92,11.72,38,0,0.0,0.0,-166.0,0.51,0.0,12.952268


### Data Profiling

In [22]:
# use pandas-profiling to generate a report on the df 
profile = df.profile_report(title='Pandas Profiling Report')
profile.to_file(output_file="EDA_weather.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
# use autoviz to make suggestions on cleaning the data
data_cleaning_suggestions(df)

Data cleaning improvement suggestions. Complete them before proceeding to ML modeling.


,Nullpercent,NuniquePercent,dtype,Nuniques,Nulls,Least num. of categories,Data cleaning improvement suggestions
LC_TEMP_QCL1,6.228492,0.096018,float64,5326,345487,0,fill missing values
LC_TEMP_QCL2,6.228492,1.661655,float64,92170,345487,0,fill missing values
LC_TEMP_QCL3,6.228492,82.348672,float64,4567782,345487,0,fill missing values
LC_HUMIDITY,5.677047,0.001586,float64,88,314899,0,fill missing values
LC_DWPTEMP,5.677047,0.064180,float64,3560,314899,0,fill missing values
LC_n,5.677047,0.000847,float64,47,314899,0,"fill missing values, left skewed distribution: cap or drop outliers"
LC_RAD,5.677047,0.016694,float64,926,314899,0,"fill missing values, right skewed distribution: cap or drop outliers"
LC_RAININ,5.677047,0.001983,float64,110,314899,0,"fill missing values, highly right skewed distribution: drop outliers or do box-cox transform"
LC_DAILYRAIN,5.677047,0.002596,float64,144,314899,0,"fill missing values, highly right skewed distribution: drop outliers or do box-cox transform"
LC_WINDDIR,5.677047,0.006490,float64,360,314899,0,fill missing values


In [5]:
# generate visualizations on the df using sweetviz
my_report = sv.analyze(df)
# save the report as an html file
my_report.show_html('EDA_weather_sweetviz.html')


                                             |          | [  0%]   00:00 -> (? left)

Report EDA_weather.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### Data Cleaning

In [54]:
# drop Year, Month, Day,Date, Hour,Minute columns from df because double info with DATEUTC
df.drop(['Year', 'Month', 'Day', 'Date', 'Hour', 'Minute'], axis=1, inplace=True)


In [55]:
# number of rows where LC_TEMP_QCL1 and LC_HUMIDITY  and LC_RAD60 are null
df[(df['LC_TEMP_QCL1'].isnull()) & (df['LC_HUMIDITY'].isnull()) & (df['LC_RAD60'].isnull())].shape

(277022, 15)

In [56]:
# data imputing not possible so we drop rows where LC_n is null
df.dropna(subset=['LC_n'], inplace=True)

In [57]:
df.shape

(5231981, 15)

In [58]:
# only keep the number part of the ID column and convert to int
df['ID'] = df['ID'].str[3:].astype(int)

In [59]:
# convert LC_HUMIDITY column to int
df['LC_HUMIDITY'] = df['LC_HUMIDITY'].astype(int)
# convert LC_n column to int
df['LC_n'] = df['LC_n'].astype(int)
# convert LC_RAD column to int
df['LC_RAD'] = df['LC_RAD'].astype(int)

In [60]:
# according to the paper, QCL3 is the most accurate data so we drop QCL0, QCL1 and QCL2
df.drop(['LC_TEMP_QCL0', 'LC_TEMP_QCL1', 'LC_TEMP_QCL2'], axis=1, inplace=True)
# rename QCL3 to LC_TEMP
df.rename(columns={'LC_TEMP_QCL3': 'LC_TEMP'}, inplace=True)

In [61]:
# save df as csv
df.to_csv('data/dataverse_files/Weather_cleaned.csv', index=False)